In [1]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:162: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [5]:
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_rtns[sect]=np.log(cleansed[d_map[sect]].resample(rule='m',how='last')).diff().dropna(how='all')

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


# Presentation to be like "Time Varying Liquidity and Momentum Profits (2015) paper

Rank by deciles, HML (8,9,10) - (1,2,3).  Save monthly returns (equal (risk?) weighted) and if you want to annualized then resample.


Then correlation matrix 
* Overall market vol?
* Market illiquidity (median Amihud through time series)


# Amihud (2002) Annual portfolio sorts

In [6]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

tsmom_model.py:275: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
tsmom_model.py:275: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


# Annual portfolio sorts

In [7]:
sector_portfolio_sorts={}
for s in sector_zscores.keys():
    sector_portfolio_sorts[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,2)
sector_portfolio_sorts['All']=quantile_portfolios_annual(data,cleansed,2)

tsmom_model.py:240: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  mkts=quantile_columns(rank_data.resample(rule='a',how='median'),year,number_of_buckets,i)
tsmom_model.py:241: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)


In [8]:
for sec in sector_portfolio_sorts.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())


Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until


\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.68 &   0.93 &    0.26 \\
Standard Deviation           &  15.17 &  15.41 &   12.98 \\
Information Ratio            &   0.53 &   0.73 &    0.24 \\
Skewness                     &  -0.08 &   0.45 &    0.02 \\
Excess Kurtosis              &   1.27 &   4.22 &    2.20 \\
CAPM Alpha Annualized (in \%) &   4.46 &   7.51 &    3.05 \\
CAPM Alpha Tstat             &   2.97 &   5.03 &    1.07 \\
CAPM Beta (in \%)             &   1.01 &   1.01 &    0.00 \\
CAPM Beta Tstat              &  25.46 &  19.00 &    0.06 \\
\$R\textasciicircum2\$                        &   0.81 &   0.79 &    0.00 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.45 &   0.62 &    0.17 \\
Standard Deviation           &   8.99 &   9.23 &    5.76 \\
Information Ratio            &   0.60 &   0.81 &    0.36 \\
Skewness            

# FHT Annual portfolio sorts

In [9]:
FHT=calculate_FHT(cleansed)
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(FHT)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

tsmom_model.py:288: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
tsmom_model.py:289: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
tsmom_model.py:290: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
tsmom_model.py:290: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')


In [10]:
sector_portfolio_sorts_FHT={}
for s in sector_zscores.keys():
    sector_portfolio_sorts_FHT[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,2)
sector_portfolio_sorts_FHT['All']=quantile_portfolios_annual(data,cleansed,2)

In [11]:
for sec in sector_portfolio_sorts_FHT.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts_FHT[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until


\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.55 &   0.65 &    0.10 \\
Standard Deviation           &  13.46 &  14.60 &   12.74 \\
Information Ratio            &   0.49 &   0.53 &    0.09 \\
Skewness                     &  -0.14 &   0.25 &    0.15 \\
Excess Kurtosis              &   1.64 &   2.68 &    1.10 \\
CAPM Alpha Annualized (in \%) &   4.93 &   6.65 &    1.72 \\
CAPM Alpha Tstat             &   3.76 &   4.58 &    0.66 \\
CAPM Beta (in \%)             &   0.97 &   1.05 &    0.08 \\
CAPM Beta Tstat              &  32.88 &  20.64 &    1.08 \\
\$R\textasciicircum2\$                        &   0.82 &   0.82 &    0.00 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.37 &   0.56 &    0.19 \\
Standard Deviation           &   7.47 &   8.69 &    6.08 \\
Information Ratio            &   0.60 &   0.78 &    0.37 \\
Skewness            

# Monthly portoflio sorts

In [12]:
amihud_monthly = read_monthly(amihud=True)

In [13]:
for sec in amihud_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(amihud_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(amihud_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.58 &   0.85 &    0.27 \\
Standard Deviation           &  14.79 &  16.38 &   12.95 \\
Information Ratio            &   0.47 &   0.62 &    0.25 \\
Skewness                     &  -0.06 &   0.37 &    0.12 \\
Excess Kurtosis              &   1.52 &   2.82 &    1.71 \\
CAPM Alpha Annualized (in \%) &   3.96 &   6.81 &    2.85 \\
CAPM Alpha Tstat             &   2.74 &   4.61 &    1.04 \\
CAPM Beta (in \%)             &   0.96 &   1.07 &    0.12 \\
CAPM Beta Tstat              &  24.73 &  19.16 &    1.36 \\
\$R\textasciicircum2\$                        &   0.80 &   0.82 &    0.01 \\
\bottomrule
\end{tabular}

All


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.37 &   0.55 &    0.17 \\
Standard Deviation           &   8.99 &   9.43 &    6.44 \\
Information Ratio            &   0.50 &   0.69 &    0.32 \\
Skewness                     &  -1.01 &  -0.72 &   -0.01 \\
Excess Kurtosis              &   5.14 &   3.12 &    0.68 \\
CAPM Alpha Annualized (in \%) &   2.65 &   4.64 &    2.00 \\
CAPM Alpha Tstat             &   3.39 &   5.71 &    1.35 \\
CAPM Beta (in \%)             &   0.96 &   1.01 &    0.05 \\
CAPM Beta Tstat              &  29.76 &  22.89 &    0.82 \\
\$R\textasciicircum2\$                        &   0.87 &   0.86 &    0.00 \\
\bottomrule
\end{tabular}

Currencies
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &  -0.01 &   0.01 &    0.02 \\
Standard Deviation           &   7.62 &   7.36 &    6.43 \\
Information Ratio            &  -0.01 &   0.02 &    0.04 \\
Skewness     

# Monthly FHT

In [14]:
FHT_monthly = read_monthly(amihud=False)

In [15]:
for sec in FHT_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(FHT_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(FHT_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())



Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.73 &   0.37 &   -0.37 \\
Standard Deviation           &  13.04 &  15.42 &   13.95 \\
Information Ratio            &   0.68 &   0.29 &   -0.32 \\
Skewness                     &   0.00 &   0.30 &    0.34 \\
Excess Kurtosis              &   1.16 &   3.91 &    2.38 \\
CAPM Alpha Annualized (in \%) &   7.98 &   2.65 &   -5.33 \\
CAPM Alpha Tstat             &   5.14 &   1.53 &   -1.68 \\
CAPM Beta (in \%)             &   0.90 &   1.11 &    0.21 \\
CAPM Beta Tstat              &  22.94 &  20.09 &    2.36 \\
\$R\textasciicircum2\$                        &   0.77 &   0.80 &    0.03 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.50 &   0.48 &   -0.02 \\
Standard Deviation           &   7.69 &   8.84 &    6.77 \\
Information Ratio            &   0.78 &   0.65 &   -0.04 \\
Skewness            

In [16]:
FHT_monthly = read_monthly(amihud=False)
sector='Currencies'
un_dec=FHT_monthly['Currencies']
sector_rtn=np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all')
    
    
un_dec['Factor']=un_dec[un_dec.columns[-1]]-un_dec[un_dec.columns[0]]
ind=un_dec['2000':.dropna(how='all').index
# AR(1) first
ex=un_dec.dropna(how='all')
en=ex.shift(-1).dropna()
en['Intercept']=1
ex=ex.ix[en.index]
r2=[]
coef=[]
tstat=[]
for i in un_dec.columns:
    res=sm.OLS(ex[str(i)],en[[str(i)]]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    coef.append(res.params[str(i)])
    tstat.append(res.tvalues[str(i)])        
ar1=pd.DataFrame()
ar1['Coef']=pd.Series(coef,index=un_dec.columns)
ar1['Tstats']=pd.Series(tstat,index=un_dec.columns)
# CAPM regression
capm_factor=pd.DataFrame()
if sector == 'All':
    capm_factor['Mkt-RF']=sector_rtn.mean(axis=1)
else:
    capm_factor['Mkt-RF']=sector_rtn[d_map[sector]].mean(axis=1)
capm_factor['Intercept']=1
alpha=[]
beta=[]
tstat_alpha=[]
tstat_beta=[]
for i in un_dec.columns:
    res=sm.OLS(un_dec.dropna()[str(i)].loc[ind],capm_factor[['Intercept','Mkt-RF']].loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    alpha.append(res.params['Intercept'])
    beta.append(res.params['Mkt-RF'])
    tstat_alpha.append(res.tvalues['Intercept'])
    tstat_beta.append(res.tvalues['Mkt-RF']) 
    r2.append(res.rsquared_adj)  
CAPM=pd.DataFrame()
CAPM['Alpha']=pd.Series(alpha,index=un_dec.columns)
CAPM['Alpha Tstat']=pd.Series(tstat_alpha,index=un_dec.columns)
CAPM['Beta']=pd.Series(beta,index=un_dec.columns)
CAPM['Beta Tstat']=pd.Series(tstat_beta,index=un_dec.columns)
CAPM['r2']=pd.Series(r2,index=un_dec.columns)



SyntaxError: invalid syntax (<ipython-input-16-5abdda202ce6>, line 8)

In [ ]:
print res.summary()

In [ ]:
un_dec.dropna()[str(i)].loc[ind]


In [ ]:
capm_factor[['Intercept','Mkt-RF']].loc[ind]

In [ ]:
def portfolio_sort_table(un_dec,sector_rtn,d_map,sector):
    un_dec['Factor']=un_dec[un_dec.columns[-1]]-un_dec[un_dec.columns[0]]
    ind=un_dec['2000':'2016'].dropna(how='all').index
    # AR(1) first
    ex=un_dec.dropna(how='all')
    en=ex.shift(-1).dropna()
    en['Intercept']=1
    ex=ex.ix[en.index]
    r2=[]
    coef=[]
    tstat=[]
    for i in un_dec.columns:
        res=sm.OLS(ex[str(i)],en[[str(i)]]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        coef.append(res.params[str(i)])
        tstat.append(res.tvalues[str(i)])        
    ar1=pd.DataFrame()
    ar1['Coef']=pd.Series(coef,index=un_dec.columns)
    ar1['Tstats']=pd.Series(tstat,index=un_dec.columns)
    # CAPM regression
    capm_factor=pd.DataFrame()
    if sector == 'All':
        capm_factor['Mkt-RF']=sector_rtn.mean(axis=1)
    else:
        capm_factor['Mkt-RF']=sector_rtn[d_map[sector]].mean(axis=1)
    capm_factor['Intercept']=1
    alpha=[]
    beta=[]
    tstat_alpha=[]
    tstat_beta=[]
    for i in un_dec.columns:
        res=sm.OLS(un_dec.dropna()[str(i)].loc[ind],capm_factor[['Intercept','Mkt-RF']].loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        alpha.append(res.params['Intercept'])
        beta.append(res.params['Mkt-RF'])
        tstat_alpha.append(res.tvalues['Intercept'])
        tstat_beta.append(res.tvalues['Mkt-RF']) 
        r2.append(res.rsquared_adj)  
    CAPM=pd.DataFrame()
    CAPM['Alpha']=pd.Series(alpha,index=un_dec.columns)
    CAPM['Alpha Tstat']=pd.Series(tstat_alpha,index=un_dec.columns)
    CAPM['Beta']=pd.Series(beta,index=un_dec.columns)
    CAPM['Beta Tstat']=pd.Series(tstat_beta,index=un_dec.columns)
    CAPM['r2']=pd.Series(r2,index=un_dec.columns)
    # Presentation
    res=pd.DataFrame()
    res['Monthly Return (in %)']=un_dec.mean()*100
    res['Standard Deviation']=un_dec.std()*math.sqrt(12)*100
    res['Information Ratio']=calc_Sharpe(un_dec)
    res['Skewness']=un_dec.skew()
    res['Excess Kurtosis']=un_dec.kurtosis()
    #res['AR(1)']=ar1.Coef
    #res['AR(1) Tstat']=ar1.Tstats
    res['CAPM Alpha Annualized (in %)']=CAPM.Alpha*1200
    res['CAPM Alpha Tstat']=CAPM['Alpha Tstat']
    res['CAPM Beta (in %)']=CAPM.Beta
    res['CAPM Beta Tstat']=CAPM['Beta Tstat']
    res['$R^2$']=CAPM.r2.abs()
    res =res.round(2)
    return res.T